# QSVM

## Quantum Kernel

The idea behind it is to encode classical data into q-bits, then computing the dot product of the resulting states.

### ZFeatureMap

For each dimension in our feature set, there is going to be a qubit that will go through a Hadamard gate and a unitary gate.

### PauliMatricerMap

Adding X, Y, or Z gates to rotate certain amount in the bloch sphere.

# Quantum Error Correction

In [1]:
pip uninstall -y qiskit qiskit-terra qiskit-ibmq-provider qiskit-aer

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install qiskit qiskit-ibmq-provider

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 20.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 39.6 MB/s eta 0:00:00
  Attempting uninstall: rustworkx
    Found existing installation: rustworkx 0.14.2
    Uninstalling rustworkx-0.14.2:
      Successfully uninstalled rustworkx-0.14.2

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from qiskit.circuit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, IBMQ
from qiskit.tools.monitor import job_monitor
from qiskit import execute

ImportError: cannot import name 'IBMQ' from 'qiskit' (/Users/fibu/anaconda3/envs/jupyterlab/lib/python3.12/site-packages/qiskit/__init__.py)

In [1]:
IBMQ.save_account('dbd98c0ff59222395dce4add827970de3827fbfd774e0c95a644ab406c02db6bc97bd7a81bd73a9140b9727a1c0bc4d9c048611892525cb82c1f607b986683ba')
IBMQ.load_account()

provider = IBMQ.get_provider(hub='ibm-q')

backend = provider.get_backend('ibmq_qasm_simulator')

q = QuantumRegister(3, 'q')
c = ClassicalRegister(1, 'c')

circuit= QuantumCircuit(q, c)

circuit.cx(q[0], q[1])
circuit.cx(q[0], q[2])
circuit.x(q[0])
circuit.cx(q[0], q[1])
circuit.cx(q[0], q[2])
circuit.ccx(q[2], q[1], q[0])
circuit.measure(q[0], c[0])

job = execute(circuit, backend, shots=1000)

job_monitor(job)
counts = job.result().get_counts()

print("\nBit flip code with error")
print()
print(counts)

ImportError: cannot import name 'QuantumRegister' from 'qiskit.circuit' (unknown location)